In [ ]:
#Importing necessary libraries
from collections import Counter
from tqdm import tqdm
import json
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import nltk as nlp
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
#Reading the data to check for values and parsing them accordingly
review_data_df = pd.read_csv('/kaggle/input/ecommerce-product-review-data/Product Review Data.csv')
review_data_df.head()

In [ ]:
#FCertain fields have NaN values which will cause trouble to filter data
review_data_df.fillna(" ",inplace = True)

In [ ]:
#Creating a combination of all the text columns
review_data_df['concat_text'] = review_data_df['reviews.text'] + ' ' + review_data_df['reviews.title'] 
review_data_df['concat_text_length'] = review_data_df.concat_text.apply(len)
review_data_df.head()

In [ ]:
#Dropping the columns that we just combined into a single column
review_data_df.drop(['colors','dateAdded','dateUpdated','dimension','ean','manufacturer','prices','reviews.date','reviews.doRecommend','reviews.numHelpful'], axis = 1, inplace = True)
review_data_df.head()

In [ ]:
#Plotting the bar graph of the ratings 
figures = plt.figure(figsize = (2, 4))
plt.bar(review_data_df["reviews.rating"].value_counts(5.0).index.tolist(),review_data_df["reviews.rating"].value_counts(1.0), color ='maroon',width = 0.4)
print(review_data_df["reviews.rating"].value_counts())

In [ ]:
#Creating the function to clean the data to exclude stopwords
lemma = WordNetLemmatizer()
def data_cleaning(text):
    text = re.sub("[^a-zA-Z]", " ", text)
    text = text.lower()                      #Conterting to lower case
    text = text.split()                      #Converting the text into stop words by split
    text = [lemma.lemmatize(word) for word in text if not word in stopwords.words('english')]    # Eliminating stopwords
    return ' '.join(text)    

In [ ]:
#Creating a function to clean the rows of all the columns
review_data_df['concat_text']=review_data_df['concat_text'].apply(text_cleaning)
review_data_df

In [ ]:
from wordcloud import WordCloud
#Wordcloud for real job postings
plt.figure(figsize = (20,20))
wc = WordCloud(width = 1600 , height = 800 , max_words = 3000).generate(" ".join(review_data_df[review_data_df.combotext == 'Great'].concat_text))
plt.imshow(wc , interpolation = 'bilinear')

In [ ]:
X = review_data_df.concat_text
y = review_data_df.review.rating

In [ ]:
#Making the train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2 , random_state = 0)

In [ ]:
#Makig use of TF-IDF and is used for retrival of string pairs

tfid = TfidfVectorizer(min_df=0, max_df=1, ngram_range=(1,3))
tfidf_train = tfid.fit_transform(X_train)
tfidf_test = tfid.transform(X_test)
print('Tfidf_train : ', tfidf_train.shape)
print('Tfidf_test : ', tfidf_test.shape)

In [ ]:
#Using NAive Bayes classifer from the library
NBC=MultinomialNB()
NB_calc=NBC.fit(tfid_train, y_train)

In [ ]:
rng = np.random.RandomState(1)
X = rng.randint(5, size=(6, 100))
y = np.array([1, 2, 3, 4, 5, 6])
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X, y)
MultinomialNB()
print(clf.predict(X[2:3]))

In [ ]:
def fit(whole_data):
    unique_words = set()
    if isinstance(whole_data, (list,)):
      for x in whole_data:
        for y in x.split():
          if len(y)<2:
            continue
          unique_words.add(y)
      unique_words = sorted(list(unique_words))
      vocab = {j:i for i,j in enumerate(unique_words)}
      Idf_values_of_all_unique_words=IDF(whole_data,unique_words)
    return vocab, Idf_values_of_all_unique_words
 Vocabulary, idf_of_vocabulary=fit(corpus) 

In [ ]:
Prediction=NBC.predict(tfid_test)

In [ ]:
#Printing the accuracy of Naive Baye's
Naive_b_accuracy=accuracy_score(y_test, Prediction)
print("Naive Bayes accuracy is :", Naive_b_accuracy)

In [ ]:
#Showing classification report
naive_b_report=metrics.classification_report(y_test, Prediction)
print("The classification report is:")
print(naive_b_report)